In [2]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from os import path

# Cargar rutas

In [4]:
rutas = {
    'k_medias': glob("resultados_experimentos/k_medias/*"),
    'gmm': glob("resultados_experimentos/gmm/*"),
    'dbscan':glob("resultados_experimentos/dbscan/*")
    }

# Graficos

In [39]:
class Graficos_DBSCAN:
    def multigrafo(self, ruta):
        df = pd.read_csv(ruta)
        x =  df['radio'].unique()
        y = df['min_vecinos'].unique()
        X, Y = np.meshgrid(x, y)
        nombre_z = df.columns.tolist()
        nombre_z.remove('radio')
        nombre_z.remove('min_vecinos')

        Zs = {}
        for nombre_z_i in nombre_z:
            Zs[nombre_z_i] = df[nombre_z_i].to_numpy().reshape(y.size, x.size)

        return X, Y, Zs

    def graficar_resultados(self, ruta):
        fig, ax = plt.subplots(2, 2, sharex='col', sharey='row')
        ax = np.ravel(ax)
        x, y, zs = self.multigrafo(ruta)
        for ax_i, z in enumerate(zs):
            im = ax[ax_i].contourf(x, y, zs[z] )
            plt.colorbar(im, ax=ax[ax_i])
            ax[ax_i].title.set_text(z)
        ax = np.reshape(ax, (2,2))
        fig.supxlabel("radio")
        fig.supylabel("numero minimo de puntos")
        fig.tight_layout()
        fig.savefig(f'graficos_experimentos/dbscan/{path.splitext(path.basename(ruta))[0]}.png')
        plt.close()


for rutas_dbscan in rutas['dbscan']:
    Graficos_DBSCAN().graficar_resultados(rutas_dbscan)

In [44]:
class Graficos_KMEDIAS:
    def multigrafo(self, ruta):
        df = pd.read_csv(ruta)
        x =  df['umbral'].unique()
        y = df['k'].unique()
        X, Y = np.meshgrid(x, y)
        nombre_z = df.columns.tolist()
        nombre_z.remove('k')
        nombre_z.remove('umbral')
        nombre_z.remove('orden')

        orden = df.orden.unique()

        Zs = {}
        for nombre_z_i in nombre_z:
            Zs[nombre_z_i] = {}
            for orden_i in orden:
                Zs[nombre_z_i][orden_i] = df[df['orden']==orden_i][nombre_z_i].to_numpy().reshape(y.size, x.size)

        return X, Y, Zs

    def graficar_resultados(self, ruta):
        # Cada fila es un orden de distancia
        # cada columna es un score
        fig, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(12,8))
        x, y, zs = self.multigrafo(ruta)
        for z_i, zdic in enumerate(zs):
            for z_j, z in enumerate(zs[zdic]):

                im = ax[z_i, z_j].contourf(x, y, zs[zdic][z] )
                plt.colorbar(im, ax=ax[z_i, z_j])
                p = z if z>0 else "$\infty$"
                titulo = f"{zdic} p={p}"
                ax[z_i, z_j].title.set_text(titulo)


        fig.supxlabel("umbral")
        fig.supylabel("k (número de centroides)")
        fig.tight_layout()

        fig.savefig(f'graficos_experimentos/k_medias/{path.splitext(path.basename(ruta))[0]}.png')
        plt.close()
    

for rutas_k_medias in rutas['k_medias']:
    Graficos_KMEDIAS().graficar_resultados(rutas_k_medias)

In [75]:
class Graficos_GMM:
    def multigrafo(self, ruta):
        df = pd.read_csv(ruta)
        x =  df['k'].unique()
        nombre_z = df.columns.tolist()
        nombre_z.remove('k')
        nombre_z.remove('epoca')

        df = df.drop("epoca",axis=1).groupby("k").mean()

        Zs = {}
        for nombre_z_i in nombre_z:
            Zs[nombre_z_i] = df[nombre_z_i].to_numpy().flatten()

        return x, Zs

    def graficar_resultados(self, ruta):
        # Cada fila es un orden de distancia
        # cada columna es un score
        fig, ax = plt.subplots(1, 3, figsize=(10,4))
        x, zs = self.multigrafo(ruta)
        for z_i, z in enumerate(zs):
            ax[z_i].plot(x, zs[z] ,'o')
            ax[z_i].title.set_text(z)

        fig.supxlabel("k (numero de clases)")
        fig.supylabel("score")
        fig.tight_layout()

        # fig.show()
        fig.savefig(f'graficos_experimentos/gmm/{path.splitext(path.basename(ruta))[0]}.png')
        plt.close()
    

for rutas_k_gmm in rutas['gmm']:
    Graficos_GMM().graficar_resultados(rutas_k_gmm)

# Obtener codigo latex

In [21]:
def escribir(ruta):
    return f""" 
\\begin{{figure}}[htbp]
\\centerline{{\\includegraphics[width=0.5\\textwidth]{{{ruta[2:]}}}}}
\\caption{{Example of a figure caption.}}
\\label{{{ruta[2:]}}}
\\end{{figure}}
"""

def obtener_latex():
    rutas = glob('./graficos_experimentos/*/*.png', recursive=True)
    with open("graficos.tex", "w") as f:
        for ruta_i in rutas:
            f.write(escribir(ruta_i))

""" 
\begin{figure}[htbp]
\centerline{\includegraphics[width=0.5\textwidth]{graficos_experimentos/dbscan/caracteristicos0_pca0.9.png}}
\caption{Example of a figure caption.}
\label{fig}
\end{figure}
"""

obtener_latex()